# Webscraping da Bolsa de Valores conforme carteira de interesse

No projeto abaixo, tentaremos fazer a raspagem dos dados das empresas cadastradas na bolsa de valores e fazer o acompanhamento períodico do serviço, com análise proveniente da carteira de negócios empresarial pretendida.
Para tanto, precisaremos de algumas bibliotecas, as quais seguem:

**1 - Instalação e download das bibliotecas**

In [5]:
!pip install pandas-datareader
!pip install pandas
!pip install altair

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


**2 - Importação das bibliotecas**

In [2]:
import pandas_datareader as pdr
import pandas as pd 
import altair as alt
import numpy as np
import datetime as dt


**3 - Criando uma função que raspa os dados de acordo com a carteira digitada**

1 - Iremos configurar as colunas obtidas

2 - Iremos eliminar dados que podem causar confusão

3 - Iremos construir um gráfico para facilitar a leitura do desempenho

4 - Iremos tornar o gráfico interativo

In [3]:
def raspagem(filtro_carteira):
    if type(filtro_carteira) == list:
        #identação
        dados_carteira = pdr.DataReader(filtros_carteira[0],
                                    data_source='yahoo',
                                    start=filtros_carteira[1],
                                    end=filtros_carteira[2]).reset_index()
        dados_carteira.rename(columns={'Adj Close':'Fechamento',
                                   'Date':'Data'}, inplace=True)
        dados_carteira['%'] = (((dados_carteira['Close'] * 100)/dados_carteira['Open'])-100).round(2)

        #################### Gráfico básico
        base = alt.Chart(dados_carteira).mark_area(
            line={'color':'darkblue'},
            color=alt.Gradient(
                gradient='linear',
                stops=[alt.GradientStop(color='white', offset=0),
                       alt.GradientStop(color='darkblue', offset=1)],
                x1=1,
                x2=1,
                y1=1,
                y2=0
            )
        ).encode(
            alt.X('Data:T'),
            alt.Y('Fechamento:Q'),
            tooltip=['Fechamento','Data','%'],



        ).properties(width=740,
                     title=carteira).interactive()

        ############################### Agregado LINHA MEDIA
        ### mudando o nome do axis ---- y=alt.Y('y', axis=alt.Axis(format='$', title='dollar amount'))
        media = alt.Chart(dados_carteira).mark_rule(color='orange').encode(
            alt.Y('average(Fechamento)', title='Média')
        )

        ############################### Agregado LINHA MÍNIMA

        min = alt.Chart(dados_carteira).mark_rule(color='black').encode(
            alt.Y('min(Fechamento)', title='Min')
        )

        ############################### Agregado LINHA MÁXIMA

        max = alt.Chart(dados_carteira).mark_rule(color='darkgreen').encode(
            alt.Y('max(Fechamento)', title='Max')
        )

        ############################### Agregado LINHA variação

        perc = alt.Chart(dados_carteira).mark_rule(color='red').encode(
            alt.Y('average(%)', title='Variação Percentual'))
        grafico = base +  min + media + max | perc
        grafico_fim = grafico.configure_title(fontSize=20,
                                font='Lato',
                                anchor='start',
                                color='Blue')

        return grafico_fim
################################################################################################# ELSE
    else:
        #identação
        dados_carteira = pdr.DataReader(conjunto_carteiras['carteiras'],
                                        data_source='yahoo',
                                        start=filtros_carteira[0],
                                        end=filtros_carteira[1]).reset_index()
        dados_carteira.rename(columns={'Adj Close':'Fechamento','Date':'Data'}, inplace=True).str()
        #dados_carteira['%'] = (((dados_carteira['Close'] * 100)/dados_carteira['Open'])-100).round(2)

        #################### Gráfico básico
            
        base = alt.Chart(dados_carteira).mark_area(
                line={'color':'darkblue'},
                color=alt.Gradient(
                    gradient='linear',
                    stops=[alt.GradientStop(color='white', offset=0),
                           alt.GradientStop(color='darkblue', offset=1)],
                    x1=1,
                    x2=1,
                    y1=1,
                    y2=0
                )
            ).encode(
                alt.X('Date:T'),
                y=alt.Y('Adj Close:Q'),  #y=alt.Y(alt.repeat('layer'), aggregate='mean', title="Mean of US and Worldwide Gross")
                tooltip=['Adj Close','Date','%'],



            ).properties(width=740,
                         title=carteira).interactive()

        ############################### Agregado LINHA MEDIA
        ### mudando o nome do axis ---- y=alt.Y('y', axis=alt.Axis(format='$', title='dollar amount'))
        media = alt.Chart(dados_carteira).mark_rule(color='orange').encode(
                alt.Y('average(Adj Close)', title='Média')
        )

        ############################### Agregado LINHA MÍNIMA

        min = alt.Chart(dados_carteira).mark_rule(color='black').encode(
                alt.Y('min(Adj Close)', title='Min')
        )

        ############################### Agregado LINHA MÁXIMA

        max = alt.Chart(dados_carteira).mark_rule(color='darkgreen').encode(
                alt.Y('max(Adj Close)', title='Max')
        )

        ############################### Agregado LINHA variação

        #perc = alt.Chart(dados_carteira).mark_rule(color='red').encode(
        #        alt.Y('average(%)', title='Variação Percentual'))
            
            
        grafico = base +  min + media + max #| perc
        grafico_fim = grafico.configure_title(fontSize=20,
                                    font='Lato',
                                    anchor='start',
                                    color='Blue')  
                    
    return grafico_fim

**4 - Inserindo os filtros da carteira para análise**

In [10]:
print('VAMOS ANALISAR ALGUMAS CARTEIRAS DA B3')
codigo = str(input('Insira o código da carteira de negócios.Para ver dados de mais de uma carteira, inclua ";" entre elas:  ')).upper().strip()
data_um = input('Data referência de INÍCIO (mm-dd-aaaa): ').strip()
data_dois = input('Data referência de FIM (mm-dd-aaaa): ').strip()

if ";" not in codigo:
    carteira = str(f'{codigo}.SA')
    data_inicio = dt.datetime.strptime(data_um, '%m-%d-%Y').date()
    data_fim =  dt.datetime.strptime(data_dois, '%m-%d-%Y').date()
    filtros_carteira = []
    filtros_carteira.append(carteira)
    filtros_carteira.append(data_inicio)
    filtros_carteira.append(data_fim)
else:
    carteira = codigo.split(';')
    data_inicio = dt.datetime.strptime(data_um, '%m-%d-%Y').date()
    data_fim =  dt.datetime.strptime(data_dois, '%m-%d-%Y').date()
    filtros_carteira = []
    filtros_carteira.append(data_inicio)
    filtros_carteira.append(data_fim)
    for item in carteira:
        posicao = carteira.index(item)
        carteira[posicao] = str(f'{item}.SA').strip()
        conjunto_carteiras = {}
        conjunto_carteiras['carteiras'] = carteira
raspagem(filtros_carteira)

VAMOS ANALISAR ALGUMAS CARTEIRAS DA B3
Insira o código da carteira de negócios.Para ver dados de mais de uma carteira, inclua ";" entre elas:  petr4
Data referência de INÍCIO (mm-dd-aaaa): 01-01-2017
Data referência de FIM (mm-dd-aaaa): 01-01-2022


alt.HConcatChart(...)

In [6]:
print('VAMOS ANALISAR ALGUMAS CARTEIRAS DA B3')
codigo = str(input('Insira o código da carteira de negócios: ')).upper().strip()
carteira = str(f'{codigo}.SA')
data_um = input('Data referência de INÍCIO (mm-dd-aaaa): ').strip()
data_inicio = dt.datetime.strptime(data_um, '%m-%d-%Y').date()
data_dois = input('Data referência de FIM (mm-dd-aaaa): ').strip()
data_fim =  dt.datetime.strptime(data_dois, '%m-%d-%Y').date()
filtros_carteira = []
filtros_carteira.append(carteira)
filtros_carteira.append(data_inicio)
filtros_carteira.append(data_fim)
raspagem(filtros_carteira)


VAMOS ANALISAR ALGUMAS CARTEIRAS DA B3
Insira o código da carteira de negócios: petr4
Data referência de INÍCIO (mm-dd-aaaa): 01-01-2010
Data referência de FIM (mm-dd-aaaa): 12-01-2022


alt.HConcatChart(...)